### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline, web_url, snippet, source, keywords, pub_date, byline, word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# URl with 'fq='
full_url = f'{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}'

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
pages = list(range(1))
for page_num in pages:

    # Create query with a page number
    # API results show 10 articles at a time
    offset = page_num * 10
    req_url = f'{full_url}&page={page_num}&offset={offset}'
    
    # Make a "GET" request and retrieve the JSON
    nyt_reviews = requests.get(req_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Check if valid status code
    if nyt_reviews['status'] == 'OK':

        # Try to save the reviews to the reviews_list
        try:

            # loop through the reviews["response"]["docs"] and append each review to the list
            for review in nyt_reviews["response"]["docs"]:
                reviews_list.append(review)
            
            # Print the page that was just retrieved
            print(f'Page {page_num} successfully retrieved.')
    
        except:
            # Print no results for page.
            print(f'Unable to save {page_num} to list.')

    else:
        # Print a message if the response content is not valid JSON
        print(f'Response status code invalid for {page_num}.')


Page 0 successfully retrieved.


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# Dump first 5 results in json format
formatted_json = json.dumps(nyt_reviews['response']['docs'][:5], indent=4)

# Print json results
print(formatted_json)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Explore values in response docs for first result
nyt_reviews['response']['docs'][:1]

[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
  'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
  'source': 'The New York Times',
  'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
   'kicker': None,
   'content_kicker': None,
   'print_headline': 'The Attachment Diaries',
   'name': None,
   'seo': None,
   'sub': None},
  'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
   {'name': 'creative_works',
    'value': 'The Attachment Diaries (Movie)',
    'rank': 2,
    'major': 'N'},
   {'name': 'persons',
    'value': 'Diment, Valentin Javier',
    'rank': 3,
    'major': 'N'}],
  'pub_date': '2023-05-25T11:00:03+0000',
  'byline': {'original': 'By Jeannette Catsoulis',
   'person': [{'firstname': 'Jeannette',
     'middlename': None,
     'lastname': 'Catsoulis',
     'qualifier': None,
     'title': None,
     'rol

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = pd.json_normalize(reviews_list)

# Display dataframe
nyt_reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Source: Hint approach from Module 6 Challenge instructions
nyt_reviews_df["title"] = nyt_reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Display dataframe
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df["keywords"] = nyt_reviews_df["keywords"].apply(extract_keywords)

In [10]:
nyt_reviews_df['keywords']

0    subject: Movies;creative_works: The Attachment...
1    subject: Movies;persons: Kapur, Shekhar;person...
2    subject: Movies;creative_works: You Can Live F...
3    subject: Movies;creative_works: A Tourist's Gu...
4    subject: Movies;persons: Zlotowski, Rebecca;cr...
5    subject: Movies;persons: Bracey, Luke (1989- )...
6    subject: Documentary Films and Programs;creati...
7    subject: Movies;creative_works: A Thousand and...
8    subject: Movies;creative_works: Your Place or ...
9    subject: Documentary Films and Programs;subjec...
Name: keywords, dtype: object

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
nyt_movie_titles = nyt_reviews_df['title'].to_list()
nyt_movie_titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl']

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [13]:
# Create an empty list to store the results
tmdb_movies = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
# Define counter i
i = 1

# Loop through the titles
for title in nyt_movie_titles:

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Construct the request URL with the title and API key
        req_url = f"{url}{title}&api_key={tmdb_api_key}"
        
        # Perform a "GET" request for The Movie Database
        tmdb_response = requests.get(f'{req_url}').json()

        # Extend the list of movies with the results from the response
        tmdb_movies.extend(tmdb_response.get('results', []))
    
        # Check if we need to sleep before making a request
        # Divide i by 50
        if i % 50 == 0:

            # Sleep every 50 requests
            time.sleep(10)

    except Exception as e:
        # Print an error message if a movie is not found or if there's another exception
        print(f'Movie "{title}" not found: {e}')

    # Add 1 to the request counter
    i += 1
    

[
    {
        "adult": false,
        "backdrop_path": null,
        "genre_ids": [
            18,
            9648,
            53,
            27
        ],
        "id": 743040,
        "original_language": "es",
        "original_title": "El apego",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.125,
        "poster_path": "/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg",
        "release_date": "2021-10-07",
        "title": "The Attachment Diaries",
        "video": false,
        "vote_average": 3.0,
        "vote_count": 4
    },
    {
        "adult": false,
        "backdrop_path": "/sClvKM8C6a1DSPzMwkKjk8

In [43]:
extra_movie_details = []

for movie in tmdb_movies:

        # Get movie id
        movie_id = movie.get("id")
        
        # Make a request for the full movie details
        full_url = f'https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}'

        # Execute "GET" request with url
        movie_detail_response = requests.get(full_url).json()
        
        # Append the full movie details to the extra_movie_details list
        extra_movie_details.append(movie_detail_response)
        
        # Extract the genre names into a list
        genres_list = []
        for movie in extra_movie_details:
            genres = movie.get('genres', [])
            for genre in genres:
                genres_list.append(genre['name'])
                
        # Extract the spoken_languages' English name into a list
        language_list = []
        for movie in extra_movie_details:
            spoken_languages = movie.get('spoken_languages', [])
            for language in spoken_languages:
                language_list.append(language['english_name'])

        # Extract the production_countries' name into a list
        prod_country_list = []
        for movie in extra_movie_details:
            production_countries = movie.get('production_countries', [])
            for country in production_countries:
                prod_country_list.append(country['name'])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        
        # Print out the title that was found

#formatted_json = json.dumps(extra_movie_details, indent=4)
#print(formatted_json)
#extra_movie_details
#movie_detail_response.items()

In [36]:
len(genres_list)

63

In [38]:
len(language_list)

49

In [40]:
len(prod_country_list)

40

In [29]:
#extra_movie_details
tmdb_movies

[{'adult': False,
  'backdrop_path': None,
  'genre_ids': [18, 9648, 53, 27],
  'id': 743040,
  'original_language': 'es',
  'original_title': 'El apego',
  'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
  'popularity': 2.125,
  'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg',
  'release_date': '2021-10-07',
  'title': 'The Attachment Diaries',
  'video': False,
  'vote_average': 3.0,
  'vote_count': 4},
 {'adult': False,
  'backdrop_path': '/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg',
  'genre_ids': [18, 10749],
  'id': 887580,
  'original_language': 'en',
  'original_title': 'You Can Live Forever',
  'overview': "When Jaime, a gay teenager, i

In [28]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# Dumps the movies obtained from TMDb API
formatted_json = json.dumps(tmdb_movies[:5], indent=4)

# Print the list of movies obtained from TMDb API
print(formatted_json)

[
    {
        "adult": false,
        "backdrop_path": null,
        "genre_ids": [
            18,
            9648,
            53,
            27
        ],
        "id": 743040,
        "original_language": "es",
        "original_title": "El apego",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.125,
        "poster_path": "/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg",
        "release_date": "2021-10-07",
        "title": "The Attachment Diaries",
        "video": false,
        "vote_average": 3.0,
        "vote_count": 4
    },
    {
        "adult": false,
        "backdrop_path": "/sClvKM8C6a1DSPzMwkKjk8

In [18]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies)

# Display df
tmdb_movies_df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,None,"[18, 9648, 53, 27]",743040,es,El apego,"Argentina, 1970s. A desperate young woman goes...",2.125,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,2021-10-07,The Attachment Diaries,False,3.000,4
1,False,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,"[18, 10749]",887580,en,You Can Live Forever,"When Jaime, a gay teenager, is sent to live in...",31.849,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,2023-03-24,You Can Live Forever,False,6.700,33
2,False,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,"[10749, 35]",813726,en,A Tourist's Guide to Love,"After an unexpected break up, a travel executi...",23.520,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,2023-04-21,A Tourist's Guide to Love,False,6.300,158
3,False,/lPVDTVYrnIBj0AwZ4NPf5TnDEfz.jpg,"[18, 35]",803694,fr,Les Enfants des autres,"Rachel loves her life, her students, her frien...",9.943,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,2022-09-21,Other People's Children,False,6.842,184
4,False,None,[10751],100744,da,Andre folks børn,"The sympathetic farmers couple, Mads and Katri...",1.197,/3vgR6GcQMIlUYeooNMGrsAVXzlD.jpg,1958-01-01,Other People's Children,False,4.000,2
5,False,/uMcqggoN9Jl95TN4vuOtUxslguA.jpg,"[10749, 18]",370725,en,Other People's Children,"A filmmaker, reeling from the death of her fam...",3.576,/urHTo7MpJxpzEaFW9rl5BQvC7Nu.jpg,2015-12-25,Other People's Children,False,4.000,8
6,False,None,[99],1149860,ru,Чужие дети,Two generations of cinematographers — fathers ...,0.600,None,2006-07-07,Other People's Children,False,0.000,0
7,False,None,[],575043,en,Other People's Children,A series of home move style shots of a mother ...,0.600,None,1978-01-01,Other People's Children,False,0.000,0
8,False,/omkxkjj0IEhzOc9CBOwK2lrmHSk.jpg,"[10749, 18, 35]",846961,en,One True Loves,Emma and Jesse are living the perfect life tog...,17.256,/6MF3JvIZkluUijw9rvDKNqPT8kC.jpg,2023-04-07,One True Loves,False,6.412,68
9,False,None,[18],893629,ru,Однолюбы,"The story is about old village residents who, ...",0.600,/sNlGmO2wWfjpCmYuOikDvXmkb8O.jpg,1983-07-13,One True Love,False,0.000,0


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
